<h1>MLTS Exercise 02 - Bayesian Linear Regression</h1>

### Task:

This notebook provides three time series datasets, each representing daily user counts for separate applications over the year 2020:
* `App_1_Users_2020.csv`
* `App_2_Users_2020.csv`
* `App_3_Users_2020.csv`

In the previous exercise, you already took a look at the datasets and did some basic visual analysis.

In this exercise, you will use Bayesian Linear Regression to analyse the datasets further and to get to know the method more in-depth. Additionally, you will use the method to do a simple forcasting on the time series.

For each dataset, complete the following tasks:

* Set up a Bayesian Linear Regression Model with PyMC on the train set and choose informative priors.
* Check if the prior is choosen well and the model can actually predict the observed train set.
* Draw samples for the posterior distribution to find the best parameters for the model.
* Check if the sampled posterior can predict the observed train data.
* Test if the model also works on the test set.


In [ ]:
# import packages
%matplotlib widget
import os
import pandas as pd
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az

In [ ]:
# Helper variables and functions
RANDOM_SEED = 42

def plot_xY(x, Y, ax):
    """This fuction plots the prediction

    Args:
        x: The date of the time series (train.index)
        Y: The prediction by the model
        ax: The matplotlib axis
    """
    quantiles = Y.quantile((0.025, 0.25, 0.5, 0.75, 0.975), dim=("chain", "draw")).transpose()

    az.plot_hdi(
        x,
        hdi_data=quantiles.sel(quantile=[0.025, 0.975]),
        fill_kwargs={"alpha": 0.25},
        smooth=False,
        ax=ax,
    )
    az.plot_hdi(
        x,
        hdi_data=quantiles.sel(quantile=[0.25, 0.75]),
        fill_kwargs={"alpha": 0.5},
        smooth=False,
        ax=ax,
    )
    ax.plot(x, quantiles.sel(quantile=0.5), color="C1", lw=3)

Load the dataset

In [ ]:
# paths to data
base_path = '../00_Datasets/01-03_App_Datasets'
# path = 'App_1_Users_2020.csv'
# path = 'App_2_Users_2020.csv'
path = 'App_3_Users_2020.csv'

path = os.path.join(base_path, path)

In [ ]:
# Read data from file
data = pd.read_csv(path, sep=';', index_col='date')
data.index = pd.to_datetime(data.index)
data.head()

Split the data into training and test sets, with the test set comprising the final three months of 2020.

In [ ]:
# Split into train and test set
split_time = '2020-10-01'
train = data[data.index < split_time]
test = data[data.index >= split_time]

In [ ]:
# Plot train and test set
fig, ax = plt.subplots(figsize=(10, 5))
ax = train["users"].plot(label='train')
test["users"].plot(label='test')
ax.axvline(split_time, c="k", ls=":")
ax.set_xlabel('Date')
ax.set_ylabel('Users')
ax.grid()
ax.set_title('Users over Time')

plt.legend()
plt.show()

Remove outliers

In [ ]:
train = train[train['users'] != 0]

## Modelling

Let us build a Bayesian Linear Regression Model

In [ ]:
# create and setup a PyMC model
with pm.Model() as model:
    # register time to the model
    tmp_time = train["time"].to_numpy()
    time = pm.Data("time", tmp_time, dims="obs_id")
    time_2 = pm.Data("time_2", tmp_time * tmp_time, dims="obs_id")
    time_3 = pm.Data("time_3", tmp_time * tmp_time * tmp_time, dims="obs_id")

    # register priors
    beta0 = pm.Normal("beta0", 6200, 500)
    beta1 = pm.Normal("beta1", 60, 20)
    beta2 = pm.Normal("beta2", -0.2, 0.2)
    beta3 = pm.Normal("beta3", 0, 0.01)

    sigma = pm.HalfNormal("sigma", 2)

    # the actual linear model
    mu = beta0 + (beta1 * time) + (beta2 * time_2) + (beta3 * time_3)

    # likelihood
    pm.Normal("obs", mu=mu, sigma=sigma, observed=train["users"].to_numpy(), dims="obs_id")

In [ ]:
# Visualize the created model
pm.model_to_graphviz(model)

## Check the prior

As part of the Bayesian workflow, we will plot our prior predictions to see what outcomes the model finds before having observed any data. Is the prior chosen well?

In [ ]:
# get the prior
with model:
    idata = pm.sample_prior_predictive(random_seed=RANDOM_SEED)

# plot the prior prediction
fig, ax = plt.subplots(figsize=(10, 5))

plot_xY(train.index, idata.prior_predictive["obs"], ax)
ax = train["users"].plot(label='observed')
ax.set_xlabel('Date')
ax.set_ylabel('Users')
ax.grid()
ax.set(title="Prior predictive distribution on train set")

plt.legend()
plt.show()

## "Train" the model

Draw samples for the posterior distribution on the train set and find the best parameters for the model.

In [ ]:
with model:
    idata.extend(pm.sample(random_seed=RANDOM_SEED, draws=2000))

In [ ]:
# Show how the model tested out different samples of parameters
az.plot_trace(idata)
plt.show()

## Posterior predictive check

Another important aspect of the Bayesian workflow is to plot the model's posterior predictions, allowing us to see how well the model can predict the observed train data.

In [ ]:
# get posterior
with model:
    idata.extend(pm.sample_posterior_predictive(idata, random_seed=RANDOM_SEED))

# plot the results
fig, ax = plt.subplots(figsize=(10, 5))

plot_xY(train.index, idata.posterior_predictive["obs"], ax)
ax = train["users"].plot(label='observed')
ax.set_xlabel('Date')
ax.set_ylabel('Users')
ax.grid()
ax.set(title="Posterior predictive distribution on train set")

plt.legend()
plt.show()

## Inference on test set

Now we will use our model to predict/forcast the expected number of users for the last 3 months of the year (test set).

In [ ]:
# set test data and predict its posterior from the model
test_time = test["time"].to_numpy()
with model:
    pm.set_data(
        {
            "time": test_time,
            "time_2": test_time * test_time,
            "time_3": test_time * test_time * test_time,
        }
    )
    test_posterior = pm.sample_posterior_predictive(
        idata, var_names=["obs"], random_seed=RANDOM_SEED
    )

In [ ]:
# plot the posterior on the test set
fig, ax = plt.subplots(figsize=(10, 5))

plot_xY(test.index, test_posterior.posterior_predictive["obs"], ax)
ax = test["users"].plot(label='observed')
ax.set_xlabel('Date')
ax.set_ylabel('Users')
ax.grid()
ax.set(title="Posterior predictive distribution on test set")

plt.legend()
plt.show()

In [ ]:
# Plot train and test set with prediction
fig, ax = plt.subplots(figsize=(10, 5))

plot_xY(train.index, idata.posterior_predictive["obs"], ax)
plot_xY(test.index, test_posterior.posterior_predictive["obs"], ax)
ax = train["users"].plot(label='train')
test["users"].plot(label='test', color='green')
ax.axvline(split_time, c="k", ls=":")
ax.set_xlabel('Date')
ax.set_ylabel('Users')
ax.grid()
ax.set_title('Users over Time')

plt.legend()
plt.show()

## Reference

This notebook is based on:
Benjamin T. Vincent . "Interrupted time series analysis". In: PyMC Examples. Ed. by PyMC Team. DOI: [10.5281/zenodo.5654871](https://doi.org/10.5281/zenodo.5654871)